In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import math, time, copy

import utils, parameters, Unet_models
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
dir_name        = "20220829_bgerr" + str(parameters.background_err)
# dir_name        = "onlyhead"
print(dir_name)

20220829_bgerr4.5


In [2]:
training_set_npz = np.load('dataset/N' + str(parameters.sigNoise) + '_training_set.npz')
x_train_obs = training_set_npz['x_train_obs']
x_train = training_set_npz['x_train']
mask_train = training_set_npz['mask_train']

x_val_obs = training_set_npz['x_val_obs']
x_val = training_set_npz['x_val']
mask_val = training_set_npz['mask_val']

stdTr = training_set_npz['std']
meanTr = training_set_npz['mean']

In [3]:
batchsize = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

training_dataset  = torch.utils.data.TensorDataset(torch.Tensor(x_train_obs), torch.Tensor(x_train), torch.Tensor(mask_train))
val_dataset       = torch.utils.data.TensorDataset(torch.Tensor(x_val_obs),  torch.Tensor(x_val), torch.Tensor(mask_val)) 

dataloaders = {
    'train': torch.utils.data.DataLoader(training_dataset, batch_size=batchsize, shuffle=True, num_workers=4, pin_memory=True),
    'val': torch.utils.data.DataLoader(val_dataset, batch_size=batchsize, shuffle=True, num_workers=4, pin_memory=True),
}

dataset_sizes = {'train': len(training_dataset), 'val': len(val_dataset)}

In [4]:
model_head = Unet_models.L63_UnetConvRec_head().to(device)
optimizer_model_head     = optim.Adam(model_head.parameters(), lr=1e-3)

In [5]:
since = time.time()

best_model_head_wts = copy.deepcopy(model_head.state_dict())
best_loss_rec   = 1e10

train_loss_rec_list = []
val_loss_rec_list = []
train_loss_dyn_list = []
val_loss_dyn_list = []
train_loss_dynbg_list = []
val_loss_dynbg_list = []
train_loss_R_list = []
val_loss_R_list = []
train_loss_I_list = []
val_loss_I_list = []

num_epochs = 20

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model_head.train()
        else:
            model_head.eval()   # Set model to evaluate mode
            
        running_loss_rec    = 0.0
        running_loss_dyn    = 0.0
        running_loss_dyn_bg = 0.0
        running_loss_dyn_gt = 0.0
        running_loss_R      = 0.0
        running_loss_I      = 0.0
        num_loss            = 0

        # Iterate over data.
        for inputs, targets, mask, in dataloaders[phase]:
            mask        = mask.to(device)
            targets     = targets.to(device)
            inputs      = inputs.to(device)
            
            optimizer_model_head.zero_grad()

            with torch.set_grad_enabled(True): 
                outputs     = model_head(inputs[:,0:1,:] * mask[:,0:1,:])
                
                loss_rec    = torch.mean((outputs - targets)**2)
                loss_dyn_bg = utils.dynamic_loss(outputs, 1, meanTr, stdTr, 3)
                loss_dyn_gt = utils.dynamic_loss(targets, 1, meanTr, stdTr, 3)
                loss_dyn    = utils.dynamic_loss(outputs, 1, meanTr, stdTr, 1)
                loss_R      = torch.sum((outputs - targets)**2 * mask) / torch.sum(mask)
                loss_I      = torch.sum((outputs - targets)**2 * (1 - mask)) / torch.sum(1 - mask)

                loss       = loss_rec

                if phase == 'train':
                    loss.backward()
                    optimizer_model_head.step()

            running_loss_rec         += loss_rec.item()    * inputs.size(0) * stdTr**2
            running_loss_dyn         += loss_dyn.item()    * inputs.size(0) * stdTr**2
            running_loss_dyn_bg      += loss_dyn_bg.item() * inputs.size(0) * stdTr**2
            running_loss_dyn_gt      += loss_dyn_gt.item() * inputs.size(0) * stdTr**2
            running_loss_R           += loss_R.item()      * inputs.size(0) * stdTr**2
            running_loss_I           += loss_I.item()      * inputs.size(0) * stdTr**2
            num_loss                 += inputs.size(0)

        epoch_loss_rec       = running_loss_rec    / num_loss
        epoch_loss_dyn       = running_loss_dyn    / num_loss
        epoch_loss_dyn_bg    = running_loss_dyn_bg / num_loss
        epoch_loss_dyn_gt    = running_loss_dyn_gt / num_loss
        epoch_loss_R         = running_loss_R      / num_loss
        epoch_loss_I         = running_loss_I      / num_loss
        
        if epoch == 0:
            print('dyn loss(gt): {:.4e}'.format(epoch_loss_dyn_gt))
        print('{} rec loss: {:.4e} dyn loss: {:.4e} dyn loss(bg): {:.4e} loss_R: {:.4e} loss_I: {:.4e}'.format(
            phase, epoch_loss_rec, epoch_loss_dyn, epoch_loss_dyn_bg, epoch_loss_R, epoch_loss_I))
        
        if phase == 'train':
            train_loss_rec_list.append(epoch_loss_rec)
            train_loss_dyn_list.append(epoch_loss_dyn)
            train_loss_dynbg_list.append(epoch_loss_dyn_bg)
            train_loss_R_list.append(epoch_loss_R)
            train_loss_I_list.append(epoch_loss_I)
        else:
            val_loss_rec_list.append(epoch_loss_rec)
            val_loss_dyn_list.append(epoch_loss_dyn)
            val_loss_dynbg_list.append(epoch_loss_dyn_bg)
            val_loss_R_list.append(epoch_loss_R)
            val_loss_I_list.append(epoch_loss_I)

        if phase == 'val' and epoch_loss_rec < best_loss_rec:
            best_loss_rec = epoch_loss_rec
            best_model_head_wts = copy.deepcopy(model_head.state_dict())

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val reconstruction loss: {:4e}'.format(best_loss_rec))

Epoch 0/19
----------


/software/python/virtualenv/py3.6-gpu/lib/python3.6/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


dyn loss(gt): 9.6957e-03
train rec loss: 1.8003e+01 dyn loss: 4.7181e-01 dyn loss(bg): 4.7227e-01 loss_R: 5.7387e+00 loss_I: 1.8470e+01
dyn loss(gt): 1.0876e-02
val rec loss: 4.0356e+00 dyn loss: 2.1364e-01 dyn loss(bg): 2.2050e-01 loss_R: 1.0273e+00 loss_I: 4.1501e+00

Epoch 1/19
----------
train rec loss: 3.4527e+00 dyn loss: 1.1928e-01 dyn loss(bg): 1.2732e-01 loss_R: 8.5049e-01 loss_I: 3.5518e+00
val rec loss: 2.8678e+00 dyn loss: 6.7117e-02 dyn loss(bg): 7.8115e-02 loss_R: 7.2849e-01 loss_I: 2.9493e+00

Epoch 2/19
----------
train rec loss: 2.7786e+00 dyn loss: 5.1175e-02 dyn loss(bg): 5.9796e-02 loss_R: 6.7856e-01 loss_I: 2.8585e+00
val rec loss: 2.8107e+00 dyn loss: 5.3871e-02 dyn loss(bg): 6.5784e-02 loss_R: 6.5367e-01 loss_I: 2.8928e+00

Epoch 3/19
----------
train rec loss: 2.4979e+00 dyn loss: 4.0938e-02 dyn loss(bg): 4.9763e-02 loss_R: 6.1222e-01 loss_I: 2.5697e+00
val rec loss: 2.3097e+00 dyn loss: 4.0507e-02 dyn loss(bg): 4.8422e-02 loss_R: 5.5904e-01 loss_I: 2.3763e+00



In [6]:
if not os.path.isdir("ckpts/" + dir_name):
    os.makedirs("ckpts/" + dir_name)

save_dir_model_head = "ckpts/" + dir_name + "/pretrain_head_epoch20"
print("saving model at " + save_dir_model_head)
torch.save(best_model_head_wts, save_dir_model_head)

saving model at ckpts/20220829_bgerr4.5/pretrain_head_epoch20


In [7]:
if not os.path.isdir("train_loss/" + dir_name):
    os.makedirs("train_loss/" + dir_name)

save_dir_loss_head  = "train_loss/" + dir_name + "/pretrain_head_epoch20"
print("saving loss at " + save_dir_loss_head)
np.savez(save_dir_loss_head,
         train_loss_rec   = train_loss_rec_list,   val_loss_rec   = val_loss_rec_list, 
         train_loss_dyn   = train_loss_dyn_list,   val_loss_dyn   = val_loss_dyn_list,
         train_loss_dynbg = train_loss_dynbg_list, val_loss_dynbg = val_loss_dynbg_list,
         train_loss_R     = train_loss_R_list,     val_loss_R     = val_loss_R_list, 
         train_loss_I     = train_loss_I_list,     val_loss_I     = val_loss_I_list,
         time = time_elapsed)

saving loss at train_loss/20220829_bgerr4.5/pretrain_head_epoch20
